In [89]:
import unicodedata
import re

def clean_text(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    #texto = re.sub(r'[^a-z0-9\s]', '', texto)  # elimina puntuación
    texto = re.sub(r'\s+', ' ', texto).strip()  # espacios extra
    return texto




In [95]:
import re

def buscar_patron(texto, etiqueta, patron):
    match = re.search(patron, texto, re.IGNORECASE)
    if match:
        return f"{etiqueta.upper()}: {match.group(0).strip()}"
    else:
        return f"{etiqueta.upper()}: No disponible"

def extract_metadata(texto):
    metadata = {}
    metadata['csj'] = buscar_patron(texto, 'csj', r'CSJ\s*[^\n\.]+')
    metadata['corte'] = buscar_patron(texto, 'corte', r'CORTE\s*[^\n\.]+')
    metadata['provincia'] = buscar_patron(texto, 'provincia', r'BUENOS AIRES\s*[^\n\.]+')
    metadata['firma'] = buscar_patron(texto, 'firma', r'FIRMADO\s*[^\n\.]+')
    metadata['parte_actora'] = buscar_patron(texto, 'parte_actora', r'PARTE ACTORA\s*[^\n\.]+')
    metadata['parte_demandada'] = buscar_patron(texto, 'parte_demandada', r'PARTE DEMANDADA\s*[^\n\.]+')
    return metadata


In [91]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(texto):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # tamaño del fragmento
        chunk_overlap=50,  # superposición entre fragmentos
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_text(texto)


In [92]:
from sentence_transformers import SentenceTransformer, util
modelo = SentenceTransformer('all-MiniLM-L6-v2')


def generate_embeddings(corpus):
    """
    Genera embeddings para un corpus de texto.
    """
    return modelo.encode(corpus, convert_to_tensor=True)


def query_similarity(consulta, corpus):
    """
    Calcula la similitud entre una consulta y un corpus de texto.
    """
    embedding_consulta = generate_embeddings(consulta)
    embeddings_corpus = generate_embeddings(corpus)
    
    scores = util.cos_sim(embedding_consulta, embeddings_corpus)
    
    return scores



In [93]:
from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_path):
    
    reader = PdfReader(pdf_path)
    texto = ""
    for page in reader.pages:
        texto += page.extract_text() + "\n"
    return texto.strip()

In [96]:
from pprint import pprint
# main
if __name__ == "__main__":
    texto=""
    """ with open('falloEj2.pdf', 'r', encoding='utf-8') as archivo:
        texto = archivo.read() """
    texto = extract_text_from_pdf('falloEj2.pdf')
    texto_limpio = clean_text(texto)
    metadatos = extract_metadata(texto_limpio)
    chunks = chunk_text(texto)
    #limpiamos cada chunk
    chunks = [clean_text(chunk) for chunk in chunks]
    corpus = chunks  # usamos los fragmentos como corpus
    embeddings_corpus = generate_embeddings(corpus)
    consulta = "Revocar sentencias"
    scores = query_similarity(consulta, corpus)

    print("Metadatos:")
    pprint(metadatos)
    # Resultado
    for i, score in enumerate(scores[0]):
        print(f"{corpus[i]} -> similitud: {score.item():.2f}")
    
        
        

Metadatos:
{'corte': 'CORTE: corte suprema de justicia de la nacion 1vistos los autos: '
          "recursos de hecho deducidos por la actora en la causa 'molina, luis "
          'omar y otros c/ estado nacional - ministerio de justicia seguridad '
          'social y derechos humanos y otro s/ personal militar y civil de las '
          'ffaa y de seg',
 'csj': 'CSJ: csj 6066/2014/rh1 y otros molina, luis omar y otros c/ estado '
        'nacional - ministerio de justicia, seguridad social y derechos '
        'humanos y otro s/ personal militar y civil de las ffaa y de seg',
 'firma': 'FIRMA: No disponible',
 'parte_actora': 'PARTE_ACTORA: No disponible',
 'parte_demandada': 'PARTE_DEMANDADA: No disponible',
 'provincia': 'PROVINCIA: buenos aires, 10 de junio de 2025 2dicten un nuevo '
              'pronunciamiento de acuerdo a lo expresado'}
csj 6066/2014/rh1 y otros molina, luis omar y otros c/ estado nacional - ministerio de justicia, seguridad social y derechos humanos y otro s